# Configuration

In [1]:
# Parameters
ENABLE_COLAB = False
PROJECT_NAME = 'ML1010-Group-Project'

#Root Machine Learning Directory. Projects appear underneath
GOOGLE_DRIVE_MOUNT = '/content/gdrive' 
COLAB_ROOT_DIR = GOOGLE_DRIVE_MOUNT + '/MyDrive/Colab Notebooks'
COLAB_INIT_DIR = COLAB_ROOT_DIR + '/utility_files'

LOCAL_ROOT_DIR = '/home/magni//ML_Root/project_root'
LOCAL_INIT_DIR = LOCAL_ROOT_DIR + '/utility_files'

START_DATA_FILE = '01_NL_ReviewText_All(new).pkl.gz'
END_DATA_FILE = '01_NL_ReviewText_All(new).pkl.gz'

##This is for experiment work only###########################
#EXPERIMENT_NAME = 'ReviewText_Lemma_Bert (XGB)'
#FILE_NAME = '01_ML1010_GP_XGB_Bert'
#LOAD_FROM_EXP = False

# Bootstrap Environment

In [2]:
#add in support for utility file directory and importing
import sys
import os

if ENABLE_COLAB:
  #Need access to drive
  from google.colab import drive
  drive.mount(GOOGLE_DRIVE_MOUNT, force_remount=True)
  
  #add in utility directory to syspath to import 
  INIT_DIR = COLAB_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = COLAB_ROOT_DIR
  
else:
  #add in utility directory to syspath to import
  INIT_DIR = LOCAL_INIT_DIR
  sys.path.append(os.path.abspath(INIT_DIR))
  
  #Config environment variables
  ROOT_DIR = LOCAL_ROOT_DIR

#Import Utility Support
from jarvis import Jarvis
jarvis = Jarvis(ROOT_DIR, PROJECT_NAME)



import mv_python_utils as mvutils


Wha...where am I?
I am awake now.

I have set your current working directory to /home/magni/ML_Root/project_root/ML1010-Group-Project
The current time is 16:55
Hello sir. Reminder, no more coffee.



# Setup Runtime Environment

In [3]:
#%%capture
if ENABLE_COLAB:
  #!pip install scipy -q
  #!pip install scikit-learn -q
  #!pip install pycaret -q
  #!pip install matplotlib -q
  #!pip install joblib -q
  #!pip install pandasql -q 
  !pip install sentence_transformers -q
  !pip install flair -q
  !pip install spacy -U -q
  !pip install spacytextblob -q
  display('Google Colab enabled')
else:
  display('Google Colab not enabled')

#Common imports
#import json
import gzip
import pandas as pd
import numpy as np
import matplotlib
import re
import nltk
import matplotlib.pyplot as plt

#!python -m spacy download en_core_web_sm

#nltk.download('stopwords')
#%matplotlib inline

'Google Colab not enabled'

# Load Data

In [4]:
### load the meta data
dataOrig = pd.read_pickle(jarvis.DATA_DIR_WORK + '/' + START_DATA_FILE)
dataOrig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63413 entries, 0 to 63412
Data columns (total 51 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   uuid                                    63413 non-null  object 
 1   reviewText                              63413 non-null  object 
 2   overall                                 63413 non-null  float64
 3   reviewText_lemma                        63413 non-null  object 
 4   reviewText_nouns                        63413 non-null  object 
 5   reviewText_adjectives                   63413 non-null  object 
 6   reviewText_verbs                        63413 non-null  object 
 7   reviewText_nav                          63413 non-null  object 
 8   reviewText_lemma_tb_pol                 63310 non-null  float64
 9   reviewText_lemma_tb_subj                63310 non-null  float64
 10  reviewText_lemma_tb_tokens              63310 non-null  fl

# Column Explosion

In [5]:
pd.set_option('mode.chained_assignment', None)

In [6]:
import cw_df_metric_utils as cwutils
import importlib
import DataPackage as dp
import DataPackageSupport as dps
import DataExperiment
import DataExperimentSupport
import time

2022-01-24 16:55:50.330722: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-24 16:55:50.330754: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
importlib.reload(cwutils)
importlib.reload(dp)
importlib.reload(dps)
importlib.reload(DataExperiment)
importlib.reload(DataExperimentSupport)

<module 'DataExperimentSupport' from '/home/magni/ML_Root/project_root/utility_files/DataExperimentSupport.py'>

In [8]:
dataOrig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63413 entries, 0 to 63412
Data columns (total 51 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   uuid                                    63413 non-null  object 
 1   reviewText                              63413 non-null  object 
 2   overall                                 63413 non-null  float64
 3   reviewText_lemma                        63413 non-null  object 
 4   reviewText_nouns                        63413 non-null  object 
 5   reviewText_adjectives                   63413 non-null  object 
 6   reviewText_verbs                        63413 non-null  object 
 7   reviewText_nav                          63413 non-null  object 
 8   reviewText_lemma_tb_pol                 63310 non-null  float64
 9   reviewText_lemma_tb_subj                63310 non-null  float64
 10  reviewText_lemma_tb_tokens              63310 non-null  fl

# 

In [10]:
#%%time
#cwutils.expandColumn(dataOrig, 'reviewText')
#cwutils.flairSentimentEncode(dataOrig, 'reviewText_lemma')
#cwutils.calcTextBlobSentiment(dataOrig, 'reviewText_lemma')
#cwutils.bertEncode(dataOrig, 'reviewText_lemma', )
#cwutils.splitSpacySentences(dataOrig, 'reviewText_lemma')

cwutils.columnEncode(data=dataOrig,
                   columnName='reviewText_adjectives',
                   #transformerType='sentence-transformers/distilbert-base-nli-mean-token',
                    transformerType = 'sentence-transformers/multi-qa-distilbert-cos-v1',
                   colSuffix='_bert')

dataOrig.to_pickle(jarvis.DATA_DIR_WORK + '/' + END_DATA_FILE)


cwutils.columnEncode(data=dataOrig,
                   columnName='reviewText_adjectives',
                   #transformerType='sentence-transformers/distilbert-base-nli-mean-token',
                    transformerType = 'sentence-transformers/multi-qa-distilbert-cos-v1',
                   colSuffix='_bert')

dataOrig.to_pickle(jarvis.DATA_DIR_WORK + '/' + END_DATA_FILE)

#cwutils.columnEncode(data=dataOrig,
#                   columnName='reviewText_lemma',
#                   transformerType='sentence-transformers/average_word_embeddings_glove.6B.300d',
#                   colSuffix='_glove')

#dataOrig.to_pickle(jarvis.DATA_DIR_WORK + '/' + END_DATA_FILE)

#cwutils.columnEncode(data=dataOrig,
#                   columnName='reviewText_lemma',
#                   transformerType='sentence-transformers/all-mpnet-base-v2',
#                   colSuffix='_mpnet')

#dataOrig.to_pickle(jarvis.DATA_DIR_WORK + '/' + END_DATA_FILE)


Loading sentence transformer: sentence-transformers/multi-qa-distilbert-cos-v1
sentence-transformers/multi-qa-distilbert-cos-v1 sentence transformer has been loaded
Generating encodings


100%|█████████████████████████████████████| 63413/63413 [12:10<00:00, 86.78it/s]


sentence-transformers/multi-qa-distilbert-cos-v1 encodings completed


# Scratchpad

In [ ]:
dataOrig.info()

# Save Files!!!!!!!!

In [ ]:
#dataOrig.to_pickle(jarvis.DATA_DIR_WORK + '/' + END_DATA_FILE)